In [62]:
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.models import load_model

In [63]:
# load the train modal ,scaler pickle , onehot
model =load_model('model.h5')

# load the encoder an scaler
with open('onehot_encoder_geo.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [64]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Female',
    'Age': 40,
    'Tenure': 5,
    'Balance': 50000,
    'NumOfProducts': 1,
    'HasCrCard': 1,
    'EstimatedSalary': 50000, 
    'IsActiveMember': 1
}

In [65]:
#one-hot encode 'Geography' 

geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\nishi\Ai learn\11-ANN Classification\ANN_venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [66]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,EstimatedSalary,IsActiveMember
0,600,France,Female,40,5,50000,1,1,50000,1


In [67]:
# encode categorical variable

input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,EstimatedSalary,IsActiveMember
0,600,France,0,40,5,50000,1,1,50000,1


In [68]:
# concatination one hot encoded data

input_df = pd.concat ([input_df.drop("Geography", axis=1), geo_encoded_df],axis =1)
input_df 

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,EstimatedSalary,IsActiveMember,Geography_France,Geography_Germany,Geography_Spain
0,600,0,40,5,50000,1,1,50000,1,1.0,0.0,0.0


In [71]:
expected_order = [
    'CreditScore', 'Gender', 'Age', 'Tenure', 'Balance',
    'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
    'Geography_France', 'Geography_Germany', 'Geography_Spain'
]
input_df = input_df.reindex(columns=expected_order, fill_value=0)



In [73]:
# scaling the input data

input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516, -1.09499335,  0.10479359, -0.00134472, -0.41792108,
        -0.91668767,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [74]:
# predection churn

prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 460ms/step


array([[0.16046199]], dtype=float32)

In [75]:
prediction_probs = prediction[0][0]
prediction_probs

0.16046199

In [76]:
if prediction_probs > 0.5:
    print("Churn")
else:
    print("Not Churn")

Not Churn
